<h2>HW3.0.
What is a merge sort? Where is it used in Hadoop?
How is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.
What is the Hadoop shuffle?</h2>

<h2>HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress)</h2>
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,
1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,
1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,
1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 



# Explore data source file structure

In [10]:
with open('Consumer_Complaints.csv') as f:
    d = list(csv.reader(f))

print (d[0])
print(d[12])

['Complaint ID', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'State', 'ZIP code', 'Submitted via', 'Date received', 'Date sent to company', 'Company', 'Company response', 'Timely response?', 'Consumer disputed?']
['1113072', 'Credit reporting', '', 'Incorrect information on credit report', 'Account status', 'NY', '11514', 'Web', '11/12/2014', '11/12/2014', 'TransUnion', 'Closed with explanation', 'Yes', '']


<h2> create folder</h2>

In [29]:
!hdfs dfs -mkdir -p /user/venamax


16/02/02 18:20:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<h2>upload source file</h2>

In [30]:
!hdfs dfs -put Consumer_Complaints.csv /user/venamax                 #### save source data file to hdfs  \
!hdfs dfs -ls

16/02/02 18:20:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 18:20:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   1 venamax supergroup   50906486 2016-02-02 18:20 Consumer_Complaints.csv


<h2>start/stop yarn  and hdfs</h2>

In [12]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh       ### start up yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh        ### start up dfs

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-venamax-resourcemanager-VENAMAX.local.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-venamax-nodemanager-VENAMAX.local.out
16/02/02 17:29:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-venamax-namenode-VENAMAX.local.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-venamax-datanode-VENAMAX.local.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-venamax-secondarynamenode-VENAMAX.local.out
16/02/02 17:29:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform.

In [ ]:
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

# Mapper

In [60]:
%%writefile mapper.py
#!/usr/bin/python
from csv import reader
import sys


for row in reader(iter(sys.stdin.readline, '')):
    product = row[1].lower()
    
    if product=='debt collection' or product == 'mortgage':
        product = product
    else:
        product = 'other'
    if product:
        sys.stderr.write("reporter:counter:complaints:,%s,1\n"%product)
    print ('%s\t%s' % (product, 1))             ### mapper out looks like 'product' \t 1


Overwriting mapper.py


In [21]:
!chmod +x mapper.py

# Reducer

In [56]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_product = None
current_count = 0
product = None
productlist = []
# input comes from STDIN
for line in sys.stdin:
    
    line = line.strip()                        ### remove leading and trailing whitespace
    line = line.split('\t')                    ### parse the input we got from mapper.py 
    product = line[0]                           ### integer generated randomly we got from mapper.py
    
    
    try:
        count = line[1]
        count = int(count)                      ### convert count (currently a string) to int

    except ValueError:                          ### if count was not a number then silently                         
        continue                                ### ignore/discard this line

                                                
                                                
    if current_product == product:                ### this IF-switch only works because Hadoop sorts map output
        current_count += count                  ### by key (here: number) before it is passed to the reducer
    else:
        if current_product:
            
            productlist.append((current_product,current_count))  ### store tuple in a list once totalize count per number

        current_count = count                   ### set current count
        current_product = product                 ### set current product


if current_product == product:                    ### do not forget to output the last word if needed!
    productlist.append((current_product,current_count))


toplist = sorted(productlist,key=lambda record: record[1], reverse=True) ### sort list from largest count to smallest

print '%25s' %'TOP Products'
print '%20s' %'Products', '%10s' %'Count'
for i in range(len(toplist)):
    print '%20s%10s' % (toplist[i][0], toplist[i][1])


Overwriting reducer.py


In [46]:
!chmod +x reducer.py

# Run Consumer_Complaints in Hadoop

<h2> remove files from prior runs </h2> 

In [61]:
!hdfs dfs -rm -r /user/venamax/complaintsOutput                           ### remove prior files

16/02/02 18:36:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 18:36:57 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/venamax/complaintsOutput


<h2> Hadoop streaming command </h2>

hadoop jar hadoopstreamingjarfile \

    -D stream.num.map.output.key.fields=n \
    -mapper mapperfile \
    -reducer reducerfile \
    -input inputfile \
    -output outputfile
    

In [62]:
!hadoop jar hadoop-*streaming*.jar -mapper mapper.py -reducer reducer.py -input Consumer_Complaints.csv -output complaintsOutput 

16/02/02 18:37:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 18:37:08 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/02 18:37:08 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/02 18:37:08 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/02 18:37:08 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/02 18:37:08 INFO mapreduce.JobSubmitter: number of splits:1
16/02/02 18:37:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local736012362_0001
16/02/02 18:37:08 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/02 18:37:08 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/02 18:37:08 INFO mapreduce.Job: Running job: job_local736012362_0001
16/02/02 18:37:08 

In [59]:
!hdfs dfs -cat complaintsOutput/part-00000

16/02/02 18:34:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
             TOP Products	
            Products      Count	
               other    142789	
            mortgage    125752	
     debt collection     44372	


<h2>=====================
END OF HW 3.1</h2>

<h2>HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters</h2>

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

# Data

In [65]:
input_file = open("oneline.txt", "w") # writing file

terms = ['foo', 'foo',  'quux',  'labs',  'foo',  'bar',  'quux' ]
b = ' '

for term in terms:
    
    input_file.write(str(term))   
    input_file.write(b)  
    
    
    
    
input_file.write('\n')

   

input_file.close()

In [66]:
!hdfs dfs -put oneline.txt /user/venamax                 #### save source data file to hdfs  \
!hdfs dfs -ls

16/02/02 18:58:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 18:58:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   1 venamax supergroup   50906486 2016-02-02 18:20 Consumer_Complaints.csv
drwxr-xr-x   - venamax supergroup          0 2016-02-02 18:37 complaintsOutput
-rw-r--r--   1 venamax supergroup         32 2016-02-02 18:58 oneline.txt


# Mapper

In [85]:
%%writefile wordcount/mapper.py
#!/usr/bin/python
import sys

sys.stderr.write("reporter:counter:Mapper Counter,Calls,1\n")

for line in sys.stdin:
    line = line.split()
    for word in line:
        print '%s\t%s' % (word,1)

Overwriting wordcount/mapper.py


In [86]:
!chmod +x mapper.py

# Reducer

In [91]:
%%writefile wordcount/reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None
words = {}

# input comes from STDIN


# input comes from STDIN
for line in sys.stdin:

    line = line.strip()                        ### remove leading and trailing whitespace
    line = line.split('\t')                    ### parse the input we got from mappe
    word = line[0]
    
    try:

        count = line[1]
        count = int(count)                      ### convert count (currently a string) to int
    except ValueError:                          ### if count was not a number then silently                         
        continue                                ### ignore/discard this line

                                                
                                                
    if current_word == word:                     ### this IF-switch only works because Hadoop sorts map output
        current_count += count                  ### by key (here: number) before it is passed to the reducer
    else:
        if current_word:
          
            words[current_word] = current_count  ### store tuple in a list once totalize count per number
  
        current_count = count                    ### set current count
        current_word = word                      ### set current word


if current_word == word:                         ### do not forget to output the last word if needed!
    words[current_word] = current_count 

for word in words:
    print 'We found "%s"' %word, ' on %s'%words[word] , 'occassions.'
    sys.stderr.write("reporter:counter:Reducer Counter,Calls,1\n") 

Overwriting wordcount/reducer.py


In [92]:
!chmod +x reducer.py

# Run onelinecount in Hadoop

<h2> remove files from prior runs</h2>

In [93]:
!hdfs dfs -rm -r /user/venamax/onelinecountOutput    

16/02/02 19:24:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 19:24:03 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/venamax/onelinecountOutput


<h2> Hadoop streaming command </h2>

In [94]:
!hadoop jar hadoop-*streaming*.jar -mapper wordcount/mapper.py -reducer wordcount/reducer.py -input oneline.txt -output onelinecountOutput

16/02/02 19:24:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/02 19:24:07 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/02 19:24:07 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/02 19:24:07 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/02 19:24:07 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/02 19:24:07 INFO mapreduce.JobSubmitter: number of splits:1
16/02/02 19:24:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local202524920_0001
16/02/02 19:24:07 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/02 19:24:07 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/02 19:24:07 INFO mapreduce.Job: Running job: job_local202524920_0001
16/02/02 19:24:07 

<h2>3.2.1 OPTIONAL 
Using 2 reducers: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). </h2>

<h2>HW3.3. Shopping Cart Analysis</h2>
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

<h2>HW3.4. (Computationally prohibitive but then again Hadoop can handle this) Pairs</h2>

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

<h2> HW3.5: Stripes
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.</h2>

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs





OPTIONAL: all HW below this are optional 

NOTE:   -- as of 1/28/2016 the instructions needs to be completed (Talk to Jimi)



== Preliminary information ===

Much of this homework beyond this point will focus on the Apriori algorithm for frequent itemset  mining and the additional step for extracting association rules from these frequent itemsets.
Please acquaint yourself with the background information (below)
before approaching the remaining  assignments.

=== Apriori background information ===

Some background material for the  Apriori algorithm is located at:

 - Slides in Live Session #3
 - https://en.wikipedia.org/wiki/Apriori_algorithm
 - https://www.dropbox.com/s/k2zm4otych279z2/Apriori-good-slides.pdf?dl=0
 - http://snap.stanford.edu/class/cs246-2014/slides/02-assocrules.pdf

Association Rules are frequently used for Market Basket Analysis (MBA) by retailers to
understand the purchase behavior of their customers. This information can be then used for
many different purposes such as cross-selling and up-selling of products, sales promotions,
loyalty programs, store design, discount plans and many others.
Evaluation of item sets: Once you have found the frequent itemsets of a dataset, you need
to choose a subset of them as your recommendations. Commonly used metrics for measuring
significance and interest for selecting rules for recommendations are: confidence; lift; and conviction.

<h2>HW3.6
What is the Apriori algorithm? Describe an example use in your domain of expertise and what kind of . Define confidence and lift.</h2>

NOTE:
For the remaining homework use the online browsing behavior dataset located at (same dataset as used above): 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 



<h2> HW3.7. Shopping Cart Analysis </h2>
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a program using the A-priori algorithm
to find products which are frequently browsed together. Fix the support to s = 100 
(i.e. product sets need to occur together at least 100 times to be considered frequent) 
and find itemsets of size 2 and 3.

Then extract association rules from these frequent items. 

A rule is of the form: 

(item1, item5) ⇒ item2.

List the top 10 discovered rules in descreasing order of confidence in the following format
 
(item1, item5) ⇒ item2, supportCount ,support, confidence


<h2> HW3.8

Benchmark your results using the pyFIM implementation of the Apriori algorithm
(Apriori - Association Rule Induction / Frequent Item Set Mining implemented by Christian Borgelt). 
You can download pyFIM from here: </h2>

http://www.borgelt.net/pyfim.html

Comment on the results from both implementations (your Hadoop MapReduce of apriori versus pyFIM) 
in terms of results and execution times.


HW3.8 (Conceptual Exercise)

Suppose that you wished to perform the Apriori algorithm once again,
though this time now with the goal of listing the top 5 rules with corresponding confidence scores 
in decreasing order of confidence score for itemsets of size 3 using Hadoop MapReduce.
A rule is now of the form: 

(item1, item2) ⇒ item3 

Recall that the Apriori algorithm is iterative for increasing itemset size,
working off of the frequent itemsets of the previous size to explore 
ONLY the NECESSARY subset of a large combinatorial space. 
Describe how you might design a framework to perform this exercise.

In particular, focus on the following:
  — map-reduce steps required
  - enumeration of item sets and filtering for frequent candidates